In [3]:
import os
import requests
import pandas as pd
import time
import re
from tqdm import tqdm

# Replace with your actual Trussed AI API key and endpoint
api_key = 'He4E9kjuR31z3NBjiE8Hjzir66Fp7KyBjCWa9qPDBXVntjUw'
api_url = 'https://fauengtrussed.fau.edu/provider/generic/chat/completions'

# Set up headers for API request
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

# Load dataset
file_path = "SF_train_cleaned.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")  # Change nrows to process more data

print(f"Total Questions in Dataset: {len(df)}")

# Function to retry API requests
def make_request_with_retry(data, wait_time=60):
    """
    Keeps retrying API requests until a successful response is received.
    """
    while True:
        response = requests.post(api_url, headers=headers, json=data)
        
        if response.status_code == 200:
            return response.json().get("choices", [{}])[0].get("message", {}).get("content", "Error: No response")
        
        print(f"⚠️ Error: {response.status_code} - {response.text}. Retrying in {wait_time} seconds...")
        time.sleep(wait_time) 
        wait_time += wait_time+60 # Wait before retrying

# Function to extract a single synthetic question from the API response
def extract_question(response_text):
    """
    Extracts a single valid question from the generated response.
    """
    questions = re.split(r"\n?\d+\.\s", response_text.strip())  # Splits numbered questions (1., 2., etc.)
    
    # Remove empty entries and unwanted text
    questions = [q.strip() for q in questions if q.strip() and "Error" not in q]
    
    return questions[0] if questions else "Error"  # Return first valid question

# Function to request a synthetic question from Trussed AI
def generate_synthetic_question(original_question, category, sub_category, mode):
    """
    Generates a synthetic question with specified difficulty mode.
    """
    difficulty_prompt = {
        "easy": "Generate a simple high school physics question that is easy to solve.",
        "medium": "Generate a standard high school physics question with moderate difficulty.",
        "hard": "Generate a challenging high school physics question that requires deep understanding."
    }
    
    conversation_history = [
        {"role": "system", "content": "You are a physics expert generating high school physics questions."},
        {"role": "user", "content": f"""
        {difficulty_prompt[mode]}
        The question should belong to the category: {category} and sub-category: {sub_category}.
        Here is an example question: {original_question}
        Generate a new question based on this example.
        """}
    ]
    
    data = {
        "model": "gpt-4o",
        "messages": conversation_history,
        "max_tokens": 300,
        "temperature": 0.8
    }

    synthetic_question = ""
    
    while not synthetic_question or synthetic_question == "Error":
        generated_text = make_request_with_retry(data)
        synthetic_question = extract_question(generated_text)

        if synthetic_question == "Error":
            print(f"⚠️ Warning: Could not generate a valid question. Retrying...")
            time.sleep(60)  # Wait before retrying

    return synthetic_question

# Function to request a synthetic answer using Chain of Thought (CoT) prompting
def generate_synthetic_answer(question, category, sub_category, mode):
    """
    Generates a synthetic answer with Chain of Thought reasoning.
    """
    conversation_history = [
        {"role": "system", "content": "You are a physics tutor providing detailed step-by-step solutions for high school physics questions."},
        {"role": "user", "content": f"""
        Provide a detailed and correct answer to the following question:
        {question}

        - The question belongs to the Category: {category} and Sub-category: {sub_category}.
        - The difficulty mode is: {mode}.
        - Think step by step before arriving at the final answer.
        - Clearly state any physics concepts or formulas used.
        - Show the calculations if applicable.
        - Ensure the answer is structured and easy to understand.
        """}
    ]
    
    data = {
        "model": "gpt-4o",
        "messages": conversation_history,
        "max_tokens": 800,
        "temperature": 0.7
    }

    return make_request_with_retry(data)

# Function to validate the synthetic answer
def validate_synthetic_answer(question, answer, category, sub_category, mode):
    """
    Uses Trussed AI to verify if the synthetic answer correctly answers the synthetic question.
    Returns a validation message: "Correct", "Incorrect", or "Unclear".
    """
    conversation_history = [
        {"role": "system", "content": "You are an AI tutor that validates physics answers."},
        {"role": "user", "content": f"""
        The question belongs to Category: {category} and Sub-category: {sub_category}.
        The difficulty mode is: {mode}.

        Does the following answer correctly and completely address the question?
        Answer only with 'Correct', 'Incorrect', or 'Unclear'.

        Question: {question}

        Answer: {answer}
        """}
    ]
    
    data = {
        "model": "gpt-4o",
        "messages": conversation_history,
        "max_tokens": 50,
        "temperature": 0.5
    }

    validation_result = ""
    
    while not validation_result or validation_result not in ["Correct", "Incorrect", "Unclear"]:
        validation_result = make_request_with_retry(data).strip()
        
        if validation_result not in ["Correct", "Incorrect", "Unclear"]:
            print(f"⚠️ Warning: Unexpected validation response: {validation_result}. Retrying...")
            time.sleep(30)  # Wait before retrying

    return validation_result

# Generate synthetic data for each question
synthetic_data = []
modes = ["easy", "medium", "hard"]

for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating Synthetic Data"):
    original_question = row["Readable_Questions"]
    category = row["Category"]
    sub_category = row["Sub-category"]

    # Generate 3 synthetic questions per mode (easy, medium, hard)
    for mode in modes:
        for i in range(3):  # 3 questions per difficulty level
            print(f"🔹 Generating {mode} question {i+1}/3 for: {original_question[:50]}...")

            # Generate synthetic question
            synthetic_question = generate_synthetic_question(original_question, category, sub_category, mode)

            # Generate synthetic answer
            synthetic_answer = generate_synthetic_answer(synthetic_question, category, sub_category, mode)

            # Validate the synthetic answer
            validation_status = validate_synthetic_answer(synthetic_question, synthetic_answer, category, sub_category, mode)

            # If validation is "Unclear" or "Incorrect", retry answer generation once
            if validation_status in ["Unclear", "Incorrect"]:
                print(f"⚠️ Validation failed ({validation_status}). Regenerating answer once...")
                synthetic_answer = generate_synthetic_answer(synthetic_question, category, sub_category, mode)
                validation_status = validate_synthetic_answer(synthetic_question, synthetic_answer, category, sub_category, mode)

            synthetic_data.append({
                "Original_Question": original_question,
                "Category": category,
                "Sub_category": sub_category,
                "Mode": mode,
                "Synthetic_Question": synthetic_question,
                "Synthetic_Answer": synthetic_answer,
                "Validation_Status": validation_status  # New Column
            })

# Convert synthetic data to DataFrame and remove empty rows
df_synthetic = pd.DataFrame(synthetic_data)
df_synthetic = df_synthetic[df_synthetic["Synthetic_Question"] != ""]  # Remove empty questions

# Save Synthetic Data
synthetic_file_path = "synthetic_physics_questions_fixed.csv"
df_synthetic.to_csv(synthetic_file_path, index=False)

display(df_synthetic)
print(f"Synthetic dataset generated and saved as {synthetic_file_path}")


Total Questions in Dataset: 951


Generating Synthetic Data:   0%|                                                | 0/951 [00:00<?, ?it/s]

🔹 Generating easy question 1/3 for: A car starts from rest at a constant acceleration ...
🔹 Generating easy question 2/3 for: A car starts from rest at a constant acceleration ...
⚠️ Validation failed (Unclear). Regenerating answer once...
🔹 Generating easy question 3/3 for: A car starts from rest at a constant acceleration ...
🔹 Generating medium question 1/3 for: A car starts from rest at a constant acceleration ...
🔹 Generating medium question 2/3 for: A car starts from rest at a constant acceleration ...
🔹 Generating medium question 3/3 for: A car starts from rest at a constant acceleration ...
🔹 Generating hard question 1/3 for: A car starts from rest at a constant acceleration ...
🔹 Generating hard question 2/3 for: A car starts from rest at a constant acceleration ...
⚠️ Validation failed (Unclear). Regenerating answer once...
🔹 Generating hard question 3/3 for: A car starts from rest at a constant acceleration ...


Generating Synthetic Data:   0%|                                    | 1/951 [03:05<48:51:26, 185.14s/it]

🔹 Generating easy question 1/3 for: From an elevated point A, a stone is projected ver...
⚠️ Validation failed (Incorrect). Regenerating answer once...
🔹 Generating easy question 2/3 for: From an elevated point A, a stone is projected ver...
🔹 Generating easy question 3/3 for: From an elevated point A, a stone is projected ver...
🔹 Generating medium question 1/3 for: From an elevated point A, a stone is projected ver...
⚠️ Validation failed (Incorrect). Regenerating answer once...
🔹 Generating medium question 2/3 for: From an elevated point A, a stone is projected ver...
⚠️ Validation failed (Incorrect). Regenerating answer once...
🔹 Generating medium question 3/3 for: From an elevated point A, a stone is projected ver...
⚠️ Validation failed (Unclear). Regenerating answer once...
🔹 Generating hard question 1/3 for: From an elevated point A, a stone is projected ver...
🔹 Generating hard question 2/3 for: From an elevated point A, a stone is projected ver...
⚠️ Validation failed (Incorr

...

🔹 Generating easy question 1/3 for: In problem (2.48), calculate the total distance tr...
🔹 Generating easy question 2/3 for: In problem (2.48), calculate the total distance tr...
🔹 Generating easy question 3/3 for: In problem (2.48), calculate the total distance tr...
⚠️ Validation failed (Unclear). Regenerating answer once...
🔹 Generating medium question 1/3 for: In problem (2.48), calculate the total distance tr...
⚠️ Validation failed (Incorrect). Regenerating answer once...
🔹 Generating medium question 2/3 for: In problem (2.48), calculate the total distance tr...
🔹 Generating medium question 3/3 for: In problem (2.48), calculate the total distance tr...
⚠️ Validation failed (Incorrect). Regenerating answer once...
🔹 Generating hard question 1/3 for: In problem (2.48), calculate the total distance tr...
⚠️ Validation failed (Unclear). Regenerating answer once...
🔹 Generating hard question 2/3 for: In problem (2.48), calculate the total distance tr...
⚠️ Validation failed (Unclear)

Generating Synthetic Data:  11%|███▌                            | 106/951 [5:09:44<29:48:32, 127.00s/it]

🔹 Generating easy question 1/3 for: In problem (2.48), calculate the height to which t...
🔹 Generating easy question 2/3 for: In problem (2.48), calculate the height to which t...
🔹 Generating easy question 3/3 for: In problem (2.48), calculate the height to which t...
🔹 Generating medium question 1/3 for: In problem (2.48), calculate the height to which t...
⚠️ Validation failed (Incorrect). Regenerating answer once...
🔹 Generating medium question 2/3 for: In problem (2.48), calculate the height to which t...
🔹 Generating medium question 3/3 for: In problem (2.48), calculate the height to which t...
⚠️ Validation failed (Unclear). Regenerating answer once...
🔹 Generating hard question 1/3 for: In problem (2.48), calculate the height to which t...
⚠️ Validation failed (Unclear). Regenerating answer once...
🔹 Generating hard question 2/3 for: In problem (2.48), calculate the height to which t...


Generating Synthetic Data:  11%|███▌                            | 106/951 [5:11:59<41:27:05, 176.60s/it]


ConnectionError: HTTPSConnectionPool(host='fauengtrussed.fau.edu', port=443): Max retries exceeded with url: /provider/generic/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x78e8935ebd30>: Failed to resolve 'fauengtrussed.fau.edu' ([Errno -3] Temporary failure in name resolution)"))